## Classifying the intervals

**NB:**

This is deprecated.

The study below corresponds to the output of the notebook "Exporting". However, in exporting, we consider steps as transitory changes in the data, and **that hypothesis was discarded** from our conversation with EDF. 

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import csv
import numpy as np
from exploitation.potential import autocovariance

suffix = "DEB1-2"
export_PATH = "../exported-datasets/"
lag = 5

with open(export_PATH+"values_"+suffix, "r") as test_f:
    all_ts = [np.array(series, dtype=np.float64) for series in list(csv.reader(test_f))]
labels = np.load(export_PATH+"labels_"+suffix+".npy")

features_ts = [] 

for ts in all_ts:
    features=[]
    features+= [np.var(ts),np.min(ts),np.max(ts),np.max(ts)-np.min(ts),np.mean(ts)]
    features+= [autocovariance(ts[:-lag],ts[lag:],True)] # True: autocorrelation
    fft = np.fft.rfft(ts)[:10] # Take the first 10 components
    assert len(fft)==10, "The fft doesn't have enough components"
    features += list(np.real(fft)) # Real part
    features += list(np.imag(fft)) # Imaginary part
    features_ts.append(features)

features_ts = np.array(features_ts)
all_concat = np.concatenate([features_ts,labels.reshape(-1,1)],axis=1)
all_concat = all_concat[~np.any(np.isnan(all_concat), axis=1)]

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

In [4]:
clf = LogisticRegression()
clf = RandomForestClassifier()

reg = Pipeline([('pca', PCA(10)), ('clf', clf)]) # PCA to keep only 10 components

X_train, X_test, y_train, y_test = train_test_split(all_concat[:,:-1], all_concat[:,[-1]], test_size=0.20, random_state=42)
reg.fit(X_train,y_train.ravel())
reg.score(X_test,y_test.ravel())

0.53140633685380767

### Doing it for each sensor

In [5]:
import pandas as pd
#os.listdir("../exported-datasets") 
sensors = ["DEB1-1","DEB1-2","DEB1-3","DEB1-4","DEB2-1","DEB2-2","DEB2-3","DEB2-4","DEB3-1","DEB3-2","DEB3-3","DEB3-4","DEB3-5","PUI-","TEM1-","TEM2-","TEM3-1","TEM3-2","TEM3-3","TEM3-4"]

In [6]:
clf = LogisticRegression()
clf = RandomForestClassifier()

reg = Pipeline([('pca', PCA(10)), ('clf', clf)]) # PCA to keep only 10 components

export_PATH = "../exported-datasets/"
lag = 5
scores = pd.DataFrame(0,index = sensors,columns=['score'])
for suffix in sensors:
    with open(export_PATH+"values_"+suffix, "r") as test_f:
        all_ts = [np.array(series, dtype=np.float64) for series in list(csv.reader(test_f))]
    labels = np.load(export_PATH+"labels_"+suffix+".npy")

    features_ts = [] 

    for ts in all_ts:
        features=[]
        features+= [np.var(ts),np.min(ts),np.max(ts),np.max(ts)-np.min(ts),np.mean(ts)]
        features+= [autocovariance(ts[:-lag],ts[lag:],True)] # True: autocorrelation
        fft = np.fft.rfft(ts)[:10] # Take the first 10 components
        assert len(fft)==10, "The fft doesn't have enough components"
        features += list(np.real(fft)) # Real part
        features += list(np.imag(fft)) # Imaginary part
        features_ts.append(features)

    features_ts = np.array(features_ts)
    all_concat = np.concatenate([features_ts,labels.reshape(-1,1)],axis=1)
    all_concat = all_concat[~np.any(np.isnan(all_concat), axis=1)]
    
    X_train, X_test, y_train, y_test = train_test_split(all_concat[:,:-1], all_concat[:,[-1]], test_size=0.20, random_state=42)
    reg.fit(X_train,y_train.ravel())
    scores.loc[suffix] = reg.score(X_test,y_test.ravel())

In [7]:
scores

,score
DEB1-1,0.490369
DEB1-2,0.540022
DEB1-3,0.544313
DEB1-4,0.566248
DEB2-1,0.574012
DEB2-2,0.600545
DEB2-3,0.580351
DEB2-4,0.580404
DEB3-1,0.570434
DEB3-2,0.579123
